# View 檢視表、視圖 

參考文章: https://www.fooish.com/sql/view.html

# View 有哪些特性

# 建立 View 

### 範例01

In [ ]:
CREATE VIEW view_example AS 
SELECT qty, price, qty*price AS value 
FROM product_table;

# 更新 View 

In [ ]:
CREATE OR REPLACE VIEW view_name [(column_list)] AS
SELECT column_name(s)
FROM table_name
WHERE condition;

# 刪除 View 

DROP VIEW view_name;

### 範例02

<img src='./img/17.png'>

In [ ]:
CREATE VIEW view_p_sum (Product, P_SUM) AS
SELECT Product, Price*Quantity 
FROM p_orders 
GROUP BY Product;

<img src='./img/18.png'>

### 範例03

In [ ]:
CREATE VIEW v_badge_count AS

WITH cte AS (
SELECT hr_prosecution_serial_id, 
       0 AS noticesystem,
	   0 AS take_off,
	   0 AS survey_information,
	   COUNT(*) AS survey_dispatch,
	   0 AS major_alarm,
	   0 AS message_alarm
FROM survey_dispatch
WHERE current_state <> 'complete'
GROUP BY hr_prosecution_serial_id

UNION ALL

SELECT hr_prosecution_serial_id, 
       0 AS noticesystem,
	   0 AS take_off,
	   COUNT(*) AS survey_information,
	   0 AS survey_dispatch,
	   0 AS major_alarm,
	   0 AS message_alarm
FROM survey_information
WHERE current_state <> 'complete'
GROUP BY hr_prosecution_serial_id

UNION ALL

SELECT hr_prosecution_serial_id,
       0 AS noticesystem,
	   COUNT(*) AS take_off,
	   0 AS survey_information,
	   0 AS survey_dispatch,
	   0 AS major_alarm,
	   0 AS message_alarm
FROM take_off_hr_prosecution
WHERE read_datetime is  null or  read_datetime = ''
GROUP BY hr_prosecution_serial_id

UNION ALL 

SELECT hr_prosecution_serial_id,
       COUNT(*) AS noticesystem,
	   0 AS take_off,
	   0 AS survey_information,
	   0 AS survey_dispatch,
	   0 AS major_alarm,
	   0 AS message_alarm
FROM noticesystem_hr_prosecution
WHERE read_datetime is  null or  read_datetime = ''
GROUP BY hr_prosecution_serial_id

UNION ALL

SELECT hr_prosecution.serial_id,
	   0 AS noticesystem,
	   0 AS take_off,
	   0 AS survey_information,
	   0 AS survey_dispatch,
	   COUNT (hr_prosecution.serial_id) AS major_alarm,
	   0 AS message_alarm
FROM v_major_alarm
JOIN hr_prosecution ON hr_prosecution.district_prosecutors_serial_id = v_major_alarm.district_prosecutors_serial_id 
WHERE current_state = 'pending' 
GROUP BY hr_prosecution.serial_id


UNION ALL

SELECT hr_prosecution_serial_id,
       0 AS noticesystem,
	   0 AS take_off,
	   0 AS survey_information,
	   0 AS survey_dispatch,
	   0 AS major_alarm,
	   COUNT(*) AS message_alarm
FROM message_alarm_hr_prosecution
WHERE read_datetime is  null or  read_datetime = ''
GROUP BY hr_prosecution_serial_id
)

SELECT  hr_prosecution_serial_id,
        sum(noticesystem) AS noticesystem,
		sum(take_off) AS take_off,
		sum(survey_information) AS survey_information,
		sum(survey_dispatch) AS survey_dispatch,
		sum(major_alarm) AS major_alarm,
		sum(message_alarm) AS message_alarm
FROM cte 
GROUP BY hr_prosecution_serial_id
GO

### 範例04

In [ ]:
CREATE VIEW [dbo].[v_major_alarm] AS SELECT
	noticesystem.serial_id,
	hr_criminal_serial_id,
	caseprj_serial_id,
	district_prosecutors_serial_id,
	message_alarm_serial_id,
	subject_type_serial_id,
	result_class.class AS category,
	result AS content,
	notice_date,
	notice_time,
	vio_flag,
	other_content AS response,
CASE
		
		WHEN subject_type_serial_id = 1 AND vio_flag <> 'P' THEN
		CASE
			WHEN ( vio_flag IS NULL OR vio_flag = '' ) THEN 'pending' 
            ELSE 'complete' 
		END 
        
        ELSE NULL 
END AS current_state,

		insdt,
		insuid,
		insunm,
		upddt,
		upduid,
		updunm 
	FROM
		noticesystem
		LEFT JOIN result_class ON result_class.serial_id = noticesystem.res_class 
WHERE
	subject_type_serial_id IN ( 1, 2 )
GO